In [17]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [71]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=1e206213b85197e5ca14613874c6d96a3c3ea83dbe73f4c9373e4b7818a9e6d1
  Stored in directory: /Users/juancho/Library/Caches/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [1]:
import os
import json
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.exceptions import RefreshError

In [3]:
import pandas as pd
from langdetect import detect, LangDetectException

In [4]:
# Definir el alcance de acceso
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
client_secrets_file = "#####" # Path al client_secret

# Autenticación manual en la consola
flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
credentials = flow.run_local_server()  # Este flujo es para Google Colab y permite autenticación manual

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=568747845022-879r6q20mfno0tcdfh13uvjvkibuf0po.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=pxBzzmwrHBQYvjGyokrtc8QcWRgxtx&access_type=offline


In [7]:
# Configura el cliente de la API de YouTube
youtube = build('youtube', 'v3', credentials=credentials)

In [9]:
# Función para obtener los nombres de las categorías de videos
def get_video_categories():
    categories = {}
    request = youtube.videoCategories().list(part="snippet", regionCode="US")
    response = request.execute()
    for item in response['items']:
        categories[item['id']] = item['snippet']['title']
    return categories

# Obtener el diccionario de categorías de videos
categories_dict = get_video_categories()
print(categories_dict)

{'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music', '15': 'Pets & Animals', '17': 'Sports', '18': 'Short Movies', '19': 'Travel & Events', '20': 'Gaming', '21': 'Videoblogging', '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', '28': 'Science & Technology', '29': 'Nonprofits & Activism', '30': 'Movies', '31': 'Anime/Animation', '32': 'Action/Adventure', '33': 'Classics', '34': 'Comedy', '35': 'Documentary', '36': 'Drama', '37': 'Family', '38': 'Foreign', '39': 'Horror', '40': 'Sci-Fi/Fantasy', '41': 'Thriller', '42': 'Shorts', '43': 'Shows', '44': 'Trailers'}


In [11]:
# Ruta del archivo Excel
filename = '######.xlsx'

try:
    existing_df = pd.read_excel(filename)
    existing_ids = set(existing_df['video_id'])
except FileNotFoundError:
    # Si el archivo no existe, crea un DataFrame vacío y un conjunto vacío de IDs
    existing_df = pd.DataFrame(columns=[
        'video_id', 'title', 'description', 'published_at', 'view_count', 'like_count', 
        'dislike_count', 'comment_count', 'tags', 'duration', 'definition', 'caption', 
        'licensed_content', 'topic_categories', 'category', 'embeddable', 'public_stats_viewable', 
        'privacy_status', 'license', 'made_for_kids', 'region', 'detected_language'
    ])
    existing_ids = set()

In [12]:
existing_df.head()

,video_id,title,description,published_at,view_count,like_count,dislike_count,comment_count,tags,duration,...,licensed_content,topic_categories,category,embeddable,public_stats_viewable,privacy_status,license,made_for_kids,region,detected_language
0,W7YT9--8IoY,"""Barbering and Fashion: HoWStyle Infiuences Ha...",NaN,2024-11-01T19:28:52Z,278846,18108.0,NaN,204.0,NaN,PT1M14S,...,True,https://en.wikipedia.org/wiki/Lifestyle_(socio...,Howto & Style,True,True,public,youtube,False,US,en
1,F3UVBS1V3U4,"Dos Días - Tito Double P, Peso Pluma (Video Of...","Dos Dias - Tito Double P, Peso Pluma\n\n ESCUC...",2024-11-01T01:00:08Z,2498811,80517.0,NaN,2011.0,"tito pp, tito doble p, tito duble pp, peso plu...",PT2M45S,...,True,"https://en.wikipedia.org/wiki/Music, https://e...",Music,True,True,public,youtube,False,US,es
2,D7F42F_JM3U,Lil Uzi Vert - Chill Bae [Official Music Video],"Lil Uzi Vert - ""Chill Bae"" \nETERNAL ATAKE 2 O...",2024-11-01T04:00:08Z,1645340,74900.0,NaN,5173.0,"Lil, Uzi, Vert, Lil Uzi Vert, Eternal Atake, M...",PT3M25S,...,True,https://en.wikipedia.org/wiki/Electronic_music...,Music,True,True,public,youtube,False,US,en
3,6CvT_1tOrlY,Fuerza Regida X Luis R Conriquez - CHOLO 7 (VI...,Cholo 7 - Ya Disponible En Todas las Plataform...,2024-11-01T00:00:05Z,1038132,47844.0,NaN,1511.0,NaN,PT2M40S,...,True,"https://en.wikipedia.org/wiki/Hip_hop_music, h...",Music,True,True,public,youtube,False,US,es
4,4ZxQ5rR8VwA,"Is This a WW2 German Spy Hideout, Discovered i...",Thanks to AG1 for sponsoring this video. Click...,2024-11-01T19:12:41Z,298814,17512.0,NaN,1715.0,NaN,PT25M12S,...,True,https://en.wikipedia.org/wiki/Lifestyle_(socio...,People & Blogs,True,True,public,youtube,False,US,en


In [15]:
region_codes = ["US", "GB", "CA", "AU", "NZ"]

all_videos = []
for region in region_codes:
    request = youtube.videos().list(
        part="snippet,statistics,contentDetails,topicDetails,status",
        chart="mostPopular",
        maxResults=1000,
        regionCode=region
    )
    response = request.execute()
    
    for item in response['items']:
        video_id = item['id']
        
        # Obtener datos del video
        title = item['snippet']['title']
        description = item['snippet'].get('description', 'N/A')
        published_at = item['snippet']['publishedAt']
        tags = ', '.join(item['snippet'].get('tags', []))  # Convierte tags a texto
        view_count = item['statistics'].get('viewCount', 'N/A')
        like_count = item['statistics'].get('likeCount', 'N/A')
        dislike_count = item['statistics'].get('dislikeCount', 'N/A')
        comment_count = item['statistics'].get('commentCount', 'N/A')
        duration = item['contentDetails'].get('duration', 'N/A')
        definition = item['contentDetails'].get('definition', 'N/A')
        caption = item['contentDetails'].get('caption', 'N/A')
        licensed_content = item['contentDetails'].get('licensedContent', 'N/A')
        topic_categories = ', '.join(item['topicDetails'].get('topicCategories', [])) if 'topicDetails' in item else 'N/A'
        category_id = item['snippet'].get('categoryId', 'N/A')
        category_name = categories_dict.get(category_id, 'N/A')
        embeddable = item['status'].get('embeddable', 'N/A')
        public_stats_viewable = item['status'].get('publicStatsViewable', 'N/A')
        privacy_status = item['status'].get('privacyStatus', 'N/A')
        license_type = item['status'].get('license', 'N/A')
        made_for_kids = item['status'].get('madeForKids', 'N/A')
        
        # Detectar el idioma del título o descripción
        try:
            detected_language = detect(title + " " + description)
        except LangDetectException:
            detected_language = "unknown"

        # Agregar todos los datos, incluyendo la región y el idioma detectado
        all_videos.append([
            video_id, title, description, published_at, view_count, like_count, 
            dislike_count, comment_count, tags, duration, definition, caption, 
            licensed_content, topic_categories, category_name, embeddable, 
            public_stats_viewable, privacy_status, license_type, made_for_kids, 
            region, detected_language
        ])


In [16]:
if all_videos:
    new_df = pd.DataFrame(all_videos, columns=[
    'video_id', 'title', 'description', 'published_at', 'view_count', 'like_count', 
    'dislike_count', 'comment_count', 'tags', 'duration', 'definition', 'caption', 
    'licensed_content', 'topic_categories', 'category', 'embeddable', 'public_stats_viewable', 
    'privacy_status', 'license', 'made_for_kids', 'region', 'detected_language'
    ])
    # Combina el nuevo DataFrame con el existente
    # Combinar los DataFrames para actualizar los datos de los videos existentes
    combined_df = pd.concat([existing_df, new_df]).drop_duplicates(subset='video_id', keep='last')

    # Guarda el DataFrame combinado en el archivo Excel
    # Guardar el DataFrame combinado
    combined_df.to_excel('########.xlsx', index=False)

    print(f"{len(combined_df)- len(existing_df)} nuevos videos guardados en '{filename}'.")
else:
    print("No hay videos nuevos para agregar.")

250 nuevos videos guardados en '/Users/juancho/Downloads/popular_videos.xlsx'.


In [19]:
combined_df.shape

(309, 22)

In [21]:
new_df.shape

(250, 22)